# Pipeline de IA com a Base de Dados do Titanic
Lista 11 - Inteligência Artificial

## 1. Importação de Bibliotecas e Carregamento da Base

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from mlxtend.frequent_patterns import apriori, association_rules

# Carregando os dados
df = pd.read_csv('https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv')
df.head()

## 2. Análise Exploratória e Pré-Processamento

In [ ]:
# Verificando informações gerais
df.info()

# Estatísticas descritivas
df.describe()

# Valores nulos
df.isnull().sum()

In [ ]:
# Preenchendo valores nulos
df['Age'].fillna(df['Age'].median(), inplace=True)
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)
df.drop(columns=['Cabin'], inplace=True)  # Muitas informações faltantes

# Extraindo título dos nomes
df['Title'] = df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
df['Title'] = df['Title'].replace(['Mlle', 'Ms'], 'Miss')
df['Title'] = df['Title'].replace(['Mme', 'Lady', 'Countess', 'Dona'], 'Mrs')
df['Title'] = df['Title'].replace(['Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer'], 'Rare')

# Criando variável "FamilySize"
df['FamilySize'] = df['SibSp'] + df['Parch'] + 1

# Codificação de variáveis categóricas
df = pd.get_dummies(df, columns=['Sex', 'Embarked', 'Title'], drop_first=True)

# Removendo colunas irrelevantes
df.drop(columns=['PassengerId', 'Name', 'Ticket'], inplace=True)

# Separando variáveis
X = df.drop('Survived', axis=1)
y = df['Survived']

# Normalizando variáveis numéricas
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

## 3. Modelagem Supervisionada (Classificação)

In [ ]:
# Dividindo os dados
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Random Forest
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

# Decision Tree
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)

# Avaliação
print("Random Forest:\n", classification_report(y_test, y_pred_rf))
print("Decision Tree:\n", classification_report(y_test, y_pred_dt))

## 4. Modelagem Não Supervisionada (Clusterização)

In [ ]:
# Aplicando PCA para visualização
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# KMeans
kmeans = KMeans(n_clusters=2, random_state=42)
clusters = kmeans.fit_predict(X_scaled)

# Visualizando
plt.figure(figsize=(10,6))
sns.scatterplot(x=X_pca[:,0], y=X_pca[:,1], hue=clusters, palette='viridis')
plt.title("Clusters de Passageiros (K-Means + PCA)")
plt.show()

## 5. Regras de Associação

In [ ]:
# Preparando dados binarizados
assoc_df = df.copy()
assoc_df['Survived'] = assoc_df['Survived'].astype(bool)

# Binarizando as variáveis
assoc_df_bin = assoc_df.astype(int).astype(bool)

# Aplicando Apriori
frequent_itemsets = apriori(assoc_df_bin, min_support=0.1, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)

# Exibindo 3 regras com maior lift
rules.sort_values(by="lift", ascending=False).head(3)

## 6. Conclusão

Com base na análise, os principais insights foram:

- O modelo Random Forest teve melhor desempenho que a árvore de decisão.
- A variável 'Title' e o tamanho da família foram relevantes na predição de sobrevivência.
- A clusterização revelou agrupamentos distintos com base no perfil dos passageiros.
- As regras de associação indicam padrões importantes, como alta sobrevivência de mulheres de 1ª classe.

O pipeline apresentado demonstra como aplicar técnicas supervisionadas, não supervisionadas e de associação em dados reais.